In [ ]:
import os

def create_folders_from_file(file_path, base_directory):
    try:
        with open(file_path, 'r') as file:
            for line in file:
                folder_name = line.strip()  # Remove leading/trailing whitespace
                folder_path = os.path.join(base_directory, folder_name)

                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)
                    print(f"Created folder: {folder_path}")
                else:
                    print(f"Folder already exists: {folder_path}")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    folder_names_file = "new_classes.txt"  # Replace with the path to your file containing folder names
    base_directory = "new_dataset"  # Replace with the path to the base directory

    create_folders_from_file(folder_names_file, base_directory)


In [11]:
"""working code"""
import urllib.request as urllib2
import json
import collections, itertools
import requests
from bs4 import BeautifulSoup
import csv
import requests
import shutil
import os

def scrap(dish, f, url, total):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        #print(soup)
        """accessing the recipe cards."""
        all_items = soup.find_all("a", {'class': 'comp mntl-card-list-items mntl-document-card mntl-card card card--no-image'})
        num=total
        #print(all_items)
        for item in all_items:
            """url of the recipe card/recipe website"""
            dish_url = item['href']
            image_url = item.find('img')['data-src']
            #print(dish_url)
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
            page = requests.get(dish_url, headers=headers)
            page.raise_for_status()
            """scrapping recipe website for ingredients"""
            dish_folder = os.path.join("new_dataset", dish)
            l = len(os.listdir(dish_folder))+1
            soup = BeautifulSoup(page.content, 'html.parser')
            web_ingredients = soup.find_all("li", {'class': 'mntl-structured-ingredients__list-item'})
            ingredients = []
            if web_ingredients and l<=50:
                for ingredient_item in web_ingredients:
                    quantity = ingredient_item.find("span", {'data-ingredient-quantity': 'true'})
                    unit = ingredient_item.find("span", {'data-ingredient-unit': 'true'})
                    name = ingredient_item.find("span", {'data-ingredient-name': 'true'})

                    formatted_ingredient = ''
                    if quantity:
                        formatted_ingredient += quantity.text.strip() + ' '
                    if unit:
                        formatted_ingredient += unit.text.strip() + ' '
                    if name:
                        formatted_ingredient += name.text.strip()
                    ingredients.append(formatted_ingredient)
                #print(ingredients)
                """scrapping recipe website for recipe steps"""
                recipe_step_elements = soup.find_all('div', class_='comp recipe__steps mntl-block')
                recipe_steps = []
                for element in recipe_step_elements:
                    p_elements = element.find_all('p',class_='comp mntl-sc-block mntl-sc-block-html')
                    for p_element in p_elements:
                        recipe_steps.append(p_element.get_text().strip())
                #print(recipe_steps)
                """scrapping recipe website images"""
                #print(image_url)
                #print(dish)
                save_image(dish, image_url)
                result = str(dish) + "," + str(dish_url) + "," + str(ingredients) + "," + str(recipe_steps) + "\n"
                f.write(result)
    except Exception as e:
        print(f'Error while scraping: {e}')
    finally:
        pass
"""def save_image(dish, image_url,num):
    resp = requests.get(image_url, stream=True)
    if not os.path.exists("dataset/" + dish):
        os.makedirs("dataset/" + dish)
    local_file = open("dataset/" + dish + "/" + str(num) + ".jpg", 'wb')
    resp.raw.decode_content = True
    shutil.copyfileobj(resp.raw, local_file)
    del resp
    return"""
def save_image(dish, image_url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
        }
        resp = requests.get(image_url, stream=True, headers=headers)
        if resp.status_code == 200:
            # Ensure the "dataset" folder exists for each dish
            dish_folder = os.path.join("new_dataset", dish)
            if not os.path.exists(dish_folder):
                os.makedirs(dish_folder)

            # Count the number of image files already saved for this dish
            num = len(os.listdir(dish_folder))+1

            # Save the image with a unique filename
            image_filename = f"{num}.jpg"
            image_path = os.path.join(dish_folder, image_filename)

            with open(image_path, 'wb') as local_file:
                resp.raw.decode_content = True
                shutil.copyfileobj(resp.raw, local_file)
        else:
            print(f"Error: Unable to download image for {dish}")
    except Exception as e:
        print(f'Error saving image: {e}')

# ... (rest of your code)

def main(page_count):
    file = open("new_classes.txt", "r")
    all_classes = file.read().split()
    count = 0
    with open('new_Recipes_capstone.txt', 'a', encoding='utf-8') as f:
        for dish in all_classes:
            total = 0
            count += 1
            print("\n\n\n\n\n" + str(count) + "\n\n\n\n")
            for i in range(1, page_count + 1):
                """url = "https://www.allrecipes.com/search?q=" + '+'.join(dish.split('_')) + "+recipe" + "&page=" + str(i)"""
                url = f"https://www.allrecipes.com/search?{'+%20'.join(dish.split('_'))}={'+%20'.join(dish.split('_'))}&offset={(i - 1) * 12}&q={'+%20'.join(dish.split('_'))}&page={i}"
                print(dish, url)
                total = scrap(dish, f, url, total)


if __name__ == "__main__":
    main(10)







1




Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=0&q=Noodles&page=1
Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=12&q=Noodles&page=2
Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=24&q=Noodles&page=3
Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=36&q=Noodles&page=4
Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=48&q=Noodles&page=5
Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=60&q=Noodles&page=6
Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=72&q=Noodles&page=7
Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=84&q=Noodles&page=8
Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=96&q=Noodles&page=9
Noodles https://www.allrecipes.com/search?Noodles=Noodles&offset=108&q=Noodles&page=10





2




Fried https://www.allrecipes.com/search?Fried=Fried&offset=0&q=Fried&page=1
Error while scraping: [WinError 3] The system canno

KeyboardInterrupt: 

In [ ]:
import os

# Function to count the number of image files in a folder
def count_images_in_folder(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg']  # Add more extensions as needed
    image_count = 0

    for filename in os.listdir(folder_path):
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            image_count += 1

    return image_count

# Read folder names from a file (one folder name per line)
folder_names_file = "new_classes.txt"  # Replace with your file name
with open(folder_names_file, "r") as file:
    folder_names = file.read().splitlines()

# Create a list of tuples containing folder names and their image counts
folder_image_counts = []

for folder_name in folder_names:
    folder_path = os.path.join("new_dataset", folder_name)  # Replace "path_to_base_folder" with the actual base folder path
    image_count = count_images_in_folder(folder_path)
    folder_image_counts.append((folder_name, image_count))

# Sort the list based on image count in descending order
folder_image_counts.sort(key=lambda x: x[1], reverse=True)
folder_with=[]
# Print folders with 50 images first and then the rest
for folder_name, image_count in folder_image_counts:
    if image_count == 50:
        print(f"Folder '{folder_name}' contains {image_count} image(s).")
        folder_with.append(folder_name)
for folder_name, image_count in folder_image_counts:
    if image_count != 50:
        print(f"Folder '{folder_name}' contains {image_count} image(s).")
print(folder_with)